> # **STILL NEED TO WORK ON THIS**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
!wget https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/input.txt

--2025-07-11 22:41:29--  https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2025-07-11 22:41:29 (12.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [27]:
class Expert(nn.Module):
  """
  Expert networkA simple MLP with a linear layer followed by a ReLU activation for each experts.
  """

  def __init__(self, embed_dim, dropout=0.1):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(embed_dim, 4*embed_dim),
        nn.ReLU(),
        nn.Linear(4*embed_dim, embed_dim),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

class NoisyTopK(nn.Module):
  """
  This is the class for the routing matrix and getting the top k experts.
  """
  def __init__(self, n_embed, n_experts, top_k):
    super().__init__()
    # This is the routing matrix which goes from embedding dim to number of experts and topk
    self.linear = nn.Linear(n_embed, n_experts)
    self.top_k = top_k

    # A bit of noise
    self.noise = nn.Linear(n_embed, n_experts)

  def forward(self, x):
    # Getting the expert selector matrix and then getting topk results from each dimensions
    logits = self.linear(x)
    noise_logits = self.noise(x)
    noisy_logits = logits + noise_logits

    topk_logits, topk_indices = torch.topk(noisy_logits, k=self.top_k, dim=2)

    # we create a same shaped matrix with all being -inf and then wherever the indices are for topk, we leave that and make others -inf
    zeros = torch.full_like(noisy_logits, float('-inf'))
    sparse_logits = zeros.scatter(-1, topk_indices, topk_logits)
    router_output = F.softmax(sparse_logits, dim=-1)

    return router_output, topk_indices

class SparseMoE(nn.Module):

  def __init__(self, embed_dim, n_experts, top_k):
    super().__init__()
    self.router = NoisyTopK(embed_dim, n_experts, top_k)
    self.experts = nn.ModuleList([Expert(embed_dim) for _ in range(n_experts)])
    self.topk = top_k

  def forward(self, x):
    gating_output , indices = self.router(x)
    final_output = torch.zeros_like(x)

    # Reshaping for batch processing
    flat_x = x.view(-1, x.size(-1)) # [batch, seq, emb] -> [batch*seq, emb]
    flat_gatting_output = gating_output.view(-1, gating_output.size(-1)) # [batch, seq, n_experts] -> [batch*seq, n_experts]

    # Processing each expert in parellel
    for i, expert in enumerate(self.experts):
      # Creating a mask where each token is routed to expert i
        # For example, expert_mask = [True, False, False, True, ...]
          # Shape: [batch, seq_len] — one True/False per token
      expert_mask = (indices == i).any(dim=-1) # [batch, seq_len]

      # Flattened to [batch * seq_len] so it matches flat_x
      flat_mask = expert_mask.view(-1) # [batch * seq_len]

      if flat_mask.any():
        # WHERVER we have TRUE in flat_mask, we take those tokens from flat_x, we pass them through expert and we save those tokens
          # At the exact places in final_output where we have true in corespondance to flat_mask
        expert_input = flat_x[flat_mask]
        expert_output = expert(expert_input)

        # Extracting and applying gating scores
        gating_scores = flat_gatting_output[flat_mask, i].unsqueeze(1)
        weighted_expert_output = gating_scores * expert_output

        # putting in weighted expert outputs to the final output matrix
        final_output[expert_mask] += weighted_expert_output.squeeze(1)

    return final_output

In [28]:
class MultiheadLatentAttention(nn.Module):
  def __init__(self, d_model, n_heads, kv_latent_dim):
    super().__init__()
    self.d_model = d_model # Dimension of embeddings
    self.n_heads = n_heads # Number of heads
    self.dh = d_model // n_heads # dimensions of heads

    self.W_q = nn.Linear(d_model, d_model) # Wuery projection
    self.W_dkv = nn.Linear(d_model, kv_latent_dim) # Down projection
    self.W_uk = nn.Linear(kv_latent_dim, d_model) # Up projection to Keys
    self.W_uv = nn.Linear(kv_latent_dim, d_model) # Up projection to values
    self.W_o = nn.Linear(d_model, d_model) # Output projection

    self.ln = nn.LayerNorm(kv_latent_dim) # Layer norm
    self.register_buffer('absorbed_k', None) # Holds W_q @ W_uk

  def forward(self, x, kv_cache=None, past_length=0):
    B, S, D = x.size()

    # Computing absorbed query once: W_q @ W_uk.T, Shape: (D, kv_latent_dim)
      # Absorbed query matrix
    if self.absorbed_k is None:
      # Matmul directly transposes the second weight matrix
      absorbed = torch.matmul(self.W_q.weight, self.W_uk.weight) # dim: (D, kv)
      self.absorbed_k = absorbed.view(self.n_heads, self.dh, -1) # (num_heads, head_dim, latent_dim)

    # Calculating kv_cache for new token
      # If we dont have kv_cache, we assign new_kv_cache to variable c_kv
    new_c_kv = self.ln(self.W_dkv(x)) # (B, S, kv_latent_dim)
    if kv_cache is None:
      c_kv = new_c_kv
    else: # If we have alod cache, we join them
      c_kv = torch.cat([kv_cache, new_c_kv], dim=1) # (B, s_full, kv_latent_dim)

    S_full = c_kv.size(1)

    # Working on values matrix
    v_full = self.W_uv(c_kv) # (B, S_full, D)
    v = v_full.view(B, S_full, self.n_heads, self.dh) # (B, S_full, num_heads, head_dim)

    # Breaking input x since W_q is absorbed
    q = x.view(B, S, self.n_heads, self.dh) # (B, S, num_heads, head_dim)

    # Computing attention scores for the last token ONLY
    attn_scores = torch.zeros(B, self.n_heads, S, S_full, device=x.device)
    # We first multiply first head of input with first head of absorbed query
      # Then we multiply the product with transpose of c_kv to get the attention scores
    for h in range(self.n_heads):
      tmp = torch.matmul(q[:, :, h, :], self.absorbed_k[h, :, :]) # (B, S, kv_latent_dim)
      attn_scores[:, h, :, :] = torch.bmm(tmp, c_kv.transpose(1,2)) # (B, S, kv_latent_dim)@(B, kv_latent_dim, s_full)=(B, S, S_full)

    attn_scores = attn_scores / (self.dh**0.5) # variance near 1
    mask = torch.tril(torch.ones((S, S_full), device=x.device), diagonal=past_length) # (S, S_full)
    attn_scores = attn_scores.masked_fill(mask.view(1, 1, S, S_full) == 0, float('-inf'))

    # Softmax on scores to get weights
    attn_weights = F.softmax(attn_scores, dim=-1)

    # Applying weights to each head o V sepratey
    out_heads = []
    for h in range(self.n_heads):
      context_h = torch.matmul(attn_weights[:, h, :, :], v[:, :, h, :])
      out_heads.append(context_h)

    # concating all the out put heads together
    out = torch.cat(out_heads, dim=-1)

    # Returning after output projection
    return self.W_o(out), c_kv

In [32]:
class Block(nn.Module):
  def __init__(self, n_embed, n_head, kv_latent_dim, n_experts, top_k):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiheadLatentAttention(n_embed, n_head, kv_latent_dim)
    self.smoe = SparseMoE(n_embed, n_experts, top_k)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    sa_output, kv_cache = self.sa(self.ln1(x))
    x = x + sa_output
    x = x + self.smoe(self.ln2(x))

    return x

In [ ]:
class We_have_deepseek_at_home(nn.Module):
    def __init__(self, emb_dim, n_head, kv_latent_dim, n_experts, top_k):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, emb_dim)
        self.pos_emb = nn.Embedding(vocab_size, emb_dim)
        self.drop_emb = nn.Dropout(0.1)

        self.trf_blocks = nn.Sequential(
            *[Block(emb_dim, n_head, kv_latent_dim, n_experts, top_k) for _ in range(12)])

        self.final_norm = LayerNorm(emb_dim)
        self.out_head = nn.Linear(
            emb_dim, vocab_size, bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits